<a href="https://colab.research.google.com/github/caoscott/nlp-final-project/blob/master/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Welcome to Colaboratory!</h1>


Colaboratory is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud.

With Colaboratory you can write and execute code, save and share your analyses, and access powerful computing resources, all for free from your browser.

To execute the code in the above cell, select it with a click and then either press the ▷ button to the left of the code, or use the keyboard shortcut "⌘/Ctrl+Enter".

All cells modify the same global state, so variables that you define by executing a cell can be used in other cells:

In [0]:
import os

if not 'train2014' in os.listdir('.'):
  !sudo apt install -f aria2
  
  !wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip -q --show-progress
  !wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Val_mscoco.zip -q --show-progress
  !wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip -q --show-progress
  !wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Val_mscoco.zip -q --show-progress
  !aria2c -x5 http://images.cocodataset.org/zips/train2014.zip 
  !aria2c -x5 http://images.cocodataset.org/zips/val2014.zip 

  !unzip -q '*.zip'

In [2]:
!rm -rf nlp-final-project
!git clone https://github.com/caoscott/nlp-final-project.git
!mv nlp-final-project/* .

Cloning into 'nlp-final-project'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 132 (delta 76), reused 90 (delta 36), pack-reused 0
Receiving objects: 100% (132/132), 16.58 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [0]:
import torch 
import torch.nn as nn
import torch.optim as optim 

import vqa

# New Section

In [0]:
criterion = nn.CrossEntropyLoss().cuda()

In [0]:
import models
model = models.FeedForward().cuda()

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [0]:
train_correct = 0
train_total = 0

In [8]:
train_loader, test_loader = vqa.get_loaders('train2014', 'val2014', 1024, 4096, 2, 2)

Read in 17615 vectors of size 300
JSON loaded.
Combined questions and answers.
Done sorting.
JSON loaded.
Combined questions and answers.
Done sorting.


In [0]:
def accuracy(loader):
  with torch.no_grad():
    total = 0
    correct = 0
    for image, question, answer in loader:
      image, question, answer = image.cuda(), question.cuda(), answer.cuda()
      out = model(image, question)
      _, predicted = torch.max(out.data, 1)
      total += answer.size(0)
      correct += predicted.eq(answer.data).sum().item()
      del image, question, answer, out, predicted
    return correct/total

In [0]:
import time

for epoch in range(0, 1):
  train_correct = 0
  train_total = 0
  train_loss = 0
  
  start_time = time.time()
  for batch_idx, (image, question, answer) in enumerate(train_loader):
    optimizer.zero_grad()
    image, question, answer = image.cuda(), question.cuda(), answer.cuda()
    
    out = model(image, question)
    loss = criterion(out, answer)
    loss.backward()
    optimizer.step()
    
    train_loss += loss.item()
    _, predicted = torch.max(out.data, 1)
    train_total += answer.size(0)
    train_correct += predicted.eq(answer.data).sum().item()
    
    del image, question, answer, out, predicted
    
    print('\r| Epoch [%3d] Iter[%3d] Time: [%.3f]'
          '\t\tLoss: %.4f Acc@1: %.3f' 
              % (epoch, batch_idx+1, time.time()-start_time,
                 loss.item(), 100.*train_correct/train_total), end='')
    
    start_time = time.time()
    
  test_acc = accuracy(test_loader)
  print('| Epoch [%3d] \tTest Acc: %.3f' % (epoch, test_acc))

| Epoch [  0] Iter[ 46] Time: [1.602]		Loss: 4.7591 Acc@1: 18.557